# Flags

In this tutorial, we will explore flags concept - a synchronization method between pipelines and nodes.

Flags are used to synchronize pipeline execution between different nodes, and stored in the netUnicorn daytabase.

Flags could be set with SetFlagTask or manually via client with the `client.set_flag_values` method.

Let's import flag-related tasks and FlagValues class:

In [1]:
import os

from netunicorn.library.tasks.flags import (
    GetFlagTask,
    AtomicDecrementFlagTask,
    AtomicIncrementFlagTask,
    SetFlagTask,
    WaitForExactFlagResultTask,
    FlagValues
)

from netunicorn.base import Pipeline, Experiment
from netunicorn.client import RemoteClient

In [2]:
# how many clients we will have
clients_count = 3
if os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611')  == 'http://localhost:26611':
    clients_count = 1

Here we will define two different pipelines - one for server node and one for clients.

Server pipeline will control flags to synchronize clients, and clients will read flags and atomically increment and decrement them to notify server about their state.

In [3]:
server_pipeline = (
    Pipeline()
    .then(SetFlagTask("clients_control", flag_values=FlagValues(text_value="stage 1", int_value=0)))
    .then(WaitForExactFlagResultTask(
        flag_name="clients_control",
        values=FlagValues(text_value="stage 1", int_value=clients_count))
    )
    .then(SetFlagTask("clients_control", flag_values=FlagValues(text_value="stage 2", int_value=clients_count)))
    .then(WaitForExactFlagResultTask(
        flag_name="clients_control",
        values=FlagValues(text_value="stage 2", int_value=0))
    )
    .then(GetFlagTask("clients_control"))
)

clients_pipeline = (
    Pipeline()
    .then(WaitForExactFlagResultTask(
        flag_name="clients_control",
        values=FlagValues(text_value="stage 1", int_value=0))
    )
    .then(AtomicIncrementFlagTask("clients_control"))
    .then(WaitForExactFlagResultTask(
        flag_name="clients_control",
        values=FlagValues(text_value="stage 2", int_value=clients_count))
    )
    .then(AtomicDecrementFlagTask("clients_control"))
    .then(GetFlagTask("clients_control"))
)

As usual, let's get access to our infrastructure

In [4]:
# if you have .env file locally for storing credentials
if '.env' in os.listdir():
    from dotenv import load_dotenv
    load_dotenv(".env")

# API connection endpoint
endpoint = os.environ.get('NETUNICORN_ENDPOINT') or 'http://localhost:26611'

# user login
login = os.environ.get('NETUNICORN_LOGIN') or 'test'

# user password
password = os.environ.get('NETUNICORN_PASSWORD') or 'test'

In [5]:
# let's create a client with these parameters
client = RemoteClient(endpoint=endpoint, login=login, password=password)
client.healthcheck()

True

We will demonstrate this tutorial on our infrastructure. If you're executing it on a different set of available nodes, please change the following code to match your infrastructure.

In [6]:
nodes = client.get_nodes()

In [7]:
client_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(clients_count)
server_node = nodes.filter(lambda node: node.name.startswith("snl-server")).take(1)

if os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611')  == 'http://localhost:26611':
    client_nodes = nodes.take(1)
    server_node = nodes.take(1)

In [8]:
print(client_nodes)
print(server_node)

[raspi-e4:5f:01:56:d8:cd, raspi-e4:5f:01:9b:85:9c, raspi-e4:5f:01:75:6b:2c]
[snl-server-5]


In [9]:
experiment = Experiment().map(server_pipeline, server_node).map(clients_pipeline, client_nodes)
experiment

 - Deployment: Node=snl-server-5, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:56:d8:cd, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:9b:85:9c, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:75:6b:2c, executor_id=, prepared=False

In [10]:
experiment_name = "flags-experiment-1"

In [11]:
if experiment_name in client.get_experiments():
    client.delete_experiment(experiment_name)

In [12]:
client.prepare_experiment(experiment, experiment_name)

'flags-experiment-1'

In [14]:
client.get_experiment_status(experiment_name)

ExperimentExecutionInformation:
status: ExperimentStatus.READY
experiment: 
 - Deployment: Node=snl-server-5, executor_id=c796c28c-dce7-466c-913a-d318dd9b15d7, prepared=True
 - Deployment: Node=raspi-e4:5f:01:56:d8:cd, executor_id=64bcc719-85d7-46e6-b3e3-ce4a8891859c, prepared=True
 - Deployment: Node=raspi-e4:5f:01:9b:85:9c, executor_id=00e3911a-adc1-43ab-8b77-2c9e6dceae55, prepared=True
 - Deployment: Node=raspi-e4:5f:01:75:6b:2c, executor_id=91e46e19-7719-4611-b166-3a9ed38cbcb4, prepared=True
execution_result:
None

In [15]:
client.start_execution(experiment_name)

'flags-experiment-1'

In [16]:
result = client.get_experiment_status(experiment_name)

In [17]:
result

ExperimentExecutionInformation:
status: ExperimentStatus.FINISHED
experiment: 
 - Deployment: Node=snl-server-5, executor_id=c796c28c-dce7-466c-913a-d318dd9b15d7, prepared=True
 - Deployment: Node=raspi-e4:5f:01:56:d8:cd, executor_id=64bcc719-85d7-46e6-b3e3-ce4a8891859c, prepared=True
 - Deployment: Node=raspi-e4:5f:01:9b:85:9c, executor_id=00e3911a-adc1-43ab-8b77-2c9e6dceae55, prepared=True
 - Deployment: Node=raspi-e4:5f:01:75:6b:2c, executor_id=91e46e19-7719-4611-b166-3a9ed38cbcb4, prepared=True
execution_result:
[DeploymentExecutionResult:
  Node: snl-server-5
  Result: <class 'returns.result.Success'>
    6429e0dc-79cb-4d83-863c-daff02416449: [<Success: 0>]
    eb4c4b51-f2a8-4882-9337-884869a72cb2: [<Success: {'text_value': 'stage 1', 'int_value': 3}>]
    942abd8d-30a5-43a4-b917-54898262022d: [<Success: 0>]
    4fe20f85-a215-43d5-8dac-dcb0c2f10878: [<Success: {'text_value': 'stage 2', 'int_value': 0}>]
    4009483d-0809-49c3-ac87-a57534654810: [<Success: text_value='stage 2' int_

In [18]:
client.get_flag_values(experiment_name, "clients_control")

FlagValues(text_value='stage 2', int_value=0)